In [14]:
import numpy as np

In [15]:
class SMSCluster(object):

    def __init__(self,smsList = []):
    
        self.smsList = smsList[:]
        self.smsAverage = np.mean(self.smsList)


def mergeClusters(clusterList):

    smsList = []
    for cluster in clusterList:
        smsList += cluster.smsList
    newCluster = SMSCluster(smsList)

    return newCluster

def relativeDistance(clusterA,clusterB):

    d = np.abs(clusterA.smsAverage-clusterB.smsAverage)/(clusterA.smsAverage+clusterB.smsAverage)
    return d


In [18]:
maxDist = 0.2
smsList = np.random.uniform(0.,1.0,1)
print(smsList)

[0.28520518]


In [19]:
clusterList = [SMSCluster([sms]) for sms in sorted(smsList)]
# Compute the distance matrix for the clusters:
dMatrix = np.full((len(clusterList),len(clusterList)),fill_value=None)
for iA,clusterA in enumerate(clusterList):
    for iB,clusterB in enumerate(clusterList):
        if iA >= iB: continue
        dMatrix[iA,iB] = relativeDistance(clusterA,clusterB)
        dMatrix[iB,iA] = dMatrix[iA,iB] # the matrix is symmetric
minDist = min([d for d in dMatrix.flatten() if d is not None ], default = maxDist)
# Merge closest cluster up to the point no more merges are possible:
while minDist < maxDist:
    # Get indices of first pair of clusters with minimum distance
    mergeIndices = np.argwhere(dMatrix == minDist)[0]
    # Merge clusters with indices in mergeIndices
    newCluster = mergeClusters(np.take(clusterList,mergeIndices))
    # If new cluster is valid (average is close to original clusters)
    # keep it. Otherwise go back to original clusters, but set
    # their distance above maxDist, so they will no longer be merged
    if True:
        # Update cluster list:
        # Remove cluster with largest index
        # and replace cluster with smallest index by new cluster
        delete_index = max(mergeIndices)
        replace_index = min(mergeIndices)
        clusterList.pop(delete_index)
        clusterList[replace_index] = newCluster 
        # Remove row and column for cluster with largest index
        dMatrix = np.delete(dMatrix,delete_index,0)
        dMatrix = np.delete(dMatrix,delete_index,1)
        # Compute the distances between the new cluster and all the
        # other ones.
        # Recompute distances using the new cluster:
        for iA,clusterA in enumerate(clusterList):
            if iA >= replace_index: continue
            dMatrix[iA,replace_index] = relativeDistance(clusterA,newCluster)
            dMatrix[replace_index,iA] = dMatrix[iA,replace_index] # the matrix is symmetric
    else:
        dMatrix[mergeIndices] = 2*maxDist
        dMatrix[np.flip(mergeIndices)] = 2*maxDist

    minDist = min([d for d in dMatrix.flatten() if d is not None ], default = maxDist)
        
print(f'{len(clusterList)} final clusters')
for ic,c in enumerate(clusterList):

    print(f'Cluster {ic}: {c.smsList}')

print('Final Matrix:',dMatrix)

ValueError: min() iterable argument is empty